In [1]:
import pandas as pd
import numpy as np
import plotly
import chart_studio.plotly as ply
import plotly.graph_objs as go

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,108-005 70 AVENUE,4.022210e+09,4052307.0,348711,4073900,4,Queens,No,No,No,...,3,180683,No,No,No,TreesCount Staff,1.027431e+06,202756.7687,Forest Hills,11375
1,147-074 7 AVENUE,4.044750e+09,4101931.0,315986,4097300,4,Queens,No,No,No,...,21,200540,No,No,No,TreesCount Staff,1.034456e+06,228644.8374,Whitestone,11357
2,390 MORGAN AVENUE,3.028870e+09,3338310.0,218365,3044900,3,Brooklyn,No,No,No,...,3,204026,No,No,No,Volunteer,1.001823e+06,200716.8913,Brooklyn,11211
3,1027 GRAND STREET,3.029250e+09,3338342.0,217969,3044900,3,Brooklyn,No,No,No,...,10,204337,No,No,No,Volunteer,1.002420e+06,199244.2531,Brooklyn,11211
4,603 6 STREET,3.010850e+09,3025654.0,223043,3016500,3,Brooklyn,No,No,No,...,21,189565,No,No,No,Volunteer,9.909138e+05,182202.4260,Brooklyn,11215
5,8 COLUMBUS AVENUE,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190422,No,No,No,Volunteer,9.884187e+05,219825.5227,New York,10023
6,120 WEST 60 STREET,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190426,No,No,No,Volunteer,9.883112e+05,219885.2785,New York,10023
7,311 WEST 50 STREET,1.010410e+09,1086093.0,103940,1012700,1,Manhattan,No,No,No,...,9,208649,No,No,No,Volunteer,9.877691e+05,217157.8561,New York,10019
8,65 JEROME AVENUE,NaN,NaN,407443,5006400,5,Staten Island,No,No,No,...,6,209610,No,No,No,TreesCount Staff,9.630732e+05,156635.5542,Staten Island,10305
9,638 AVENUE Z,3.072350e+09,3320727.0,207508,3037402,3,Brooklyn,No,No,No,...,21,192755,No,No,No,TreesCount Staff,9.926537e+05,152903.6306,Brooklyn,11223


Looks good, but lets take a look at the shape of this data:

In [3]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [4]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,108-005 70 AVENUE,4022210001,4052307,348711,4073900,4,Queens,No,No,No,...,3,180683,No,No,No,TreesCount Staff,1027431.148,202756.7687,Forest Hills,11375
1,147-074 7 AVENUE,4044750045,4101931,315986,4097300,4,Queens,No,No,No,...,21,200540,No,No,No,TreesCount Staff,1034455.701,228644.8374,Whitestone,11357
2,390 MORGAN AVENUE,3028870001,3338310,218365,3044900,3,Brooklyn,No,No,No,...,3,204026,No,No,No,Volunteer,1001822.831,200716.8913,Brooklyn,11211
3,1027 GRAND STREET,3029250001,3338342,217969,3044900,3,Brooklyn,No,No,No,...,10,204337,No,No,No,Volunteer,1002420.358,199244.2531,Brooklyn,11211
4,603 6 STREET,3010850052,3025654,223043,3016500,3,Brooklyn,No,No,No,...,21,189565,No,No,No,Volunteer,990913.775,182202.4260,Brooklyn,11215


If we wanted the next 5, we would use this url:

In [5]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,address,bbl,bin,block_id,boro_ct,borocode,boroname,brch_light,brch_other,brch_shoe,...,tree_dbh,tree_id,trnk_light,trnk_other,trunk_wire,user_type,x_sp,y_sp,zip_city,zipcode
0,8 COLUMBUS AVENUE,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190422,No,No,No,Volunteer,988418.6997,219825.5227,New York,10023
1,120 WEST 60 STREET,1.011310e+09,1076229.0,106099,1014500,1,Manhattan,No,No,No,...,11,190426,No,No,No,Volunteer,988311.1900,219885.2785,New York,10023
2,311 WEST 50 STREET,1.010410e+09,1086093.0,103940,1012700,1,Manhattan,No,No,No,...,9,208649,No,No,No,Volunteer,987769.1163,217157.8561,New York,10019
3,65 JEROME AVENUE,NaN,NaN,407443,5006400,5,Staten Island,No,No,No,...,6,209610,No,No,No,TreesCount Staff,963073.1998,156635.5542,Staten Island,10305
4,638 AVENUE Z,3.072350e+09,3320727.0,207508,3037402,3,Brooklyn,No,No,No,...,21,192755,No,No,No,TreesCount Staff,992653.7253,152903.6306,Brooklyn,11223


You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [14]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

#print(soql_url)
#soql_trees

In [6]:
data = soql_trees

In [11]:
data['count_tree_id'].sum()

85203

In [15]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Brooklyn\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)


In [17]:
soql_trees['count_tree_id'].sum()

177293

This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [10]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [ ]:
Questio 1